<a href="https://colab.research.google.com/github/voshna123/tensorflow_practice/blob/main/foodrecog_big_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-01-09 04:01:13--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-01-09 04:01:13 (111 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves,compare_historys,unzip_data

## Loading 101Food Dataset

In [3]:
import tensorflow_datasets as tfds

In [ ]:
(train_data, test_data),ds_info = tfds.load(name = 'food101',
                                            split =['train', 'validation'],
                                            shuffle_files = True,
                                            as_supervised = True,
                                            with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
class_names = ds_info.features['label'].names
class_names[:10]

In [ ]:
train_one_sample = train_data.take(1)
train_one_sample

In [ ]:
for img, label in train_one_sample:
    print(f'Image shape: {img.shape}')
    print(f'Image label: {label}')
    print(f'Class name of label: {class_names[label.numpy()]}')

In [ ]:
img

## plot the img

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img.numpy())
plt.title(class_names[label.numpy()])

## Preprocessing data

In [ ]:
def preprocessing_Data(img, label, img_shape = 224):
    img = tf.image.resize(img, [img_shape, img_shape])
    img = tf.cast(img, tf.float32)
    return img, label